In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('customers.csv')

In [4]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


We are interested in the lifetime value of loyal customers. Thus, we can ignore (for now) those who leave the company (churn). Hence we drop the Churn column.

In [9]:
data.drop(columns='Churn',inplace=True)

In [22]:
data.drop(columns='customerID',inplace=True)

In [97]:
data['TotalCharges'] = data['TotalCharges'].replace(" ", np.nan).astype('float32')
#drop Nan
data.dropna(how='any', axis=0,inplace=True)

As we are intrested in the lifetime value of the contracts, we can compute the average monthly expense of each customer, then the lifetime value of the contracts could be extrapolate from the average lifetime of the contracts.

In [24]:
data['MonthlyExpense'] = data['TotalCharges']/data['tenure']

Now we would be interested in knowing how these average monthly expenses are explain by the other features. But first we need to do some feature engineering. Indeed as we have seen during de preliminary data inspection, we have a lot of categorical feature that we will need to map to numerical value. When it is a binary value, e.g. yes/no or male/female we can directly replace it by 0 or I. When there is more possibility like for the 'contract' feature we will use the pd.factorize() function to establish a numerical mapping. Furthermore, notice that some feature present redundant information such as the 'No phone service' information that is already present in the 'PhoneService' feature and the 'No internet service' present in all the internet related services features that could be placed in a separated feature: 'InternetService' under the form of true or false (an additional column will be created for 'Fiber'->True/False). Proceeding this way most of the categorical feature will become binary except 'contract' and 'payement method'.

In [25]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float32
MonthlyExpense      float64
dtype: object

In [27]:
for column in data.select_dtypes(include=np.object).columns:

    print(column)
    print(set(data[column]))

gender
{'Male', 'Female'}
Partner
{'Yes', 'No'}
Dependents
{'Yes', 'No'}
PhoneService
{'Yes', 'No'}
MultipleLines
{'No phone service', 'Yes', 'No'}
InternetService
{'DSL', 'Fiber optic', 'No'}
OnlineSecurity
{'No internet service', 'Yes', 'No'}
OnlineBackup
{'No internet service', 'Yes', 'No'}
DeviceProtection
{'No internet service', 'Yes', 'No'}
TechSupport
{'No internet service', 'Yes', 'No'}
StreamingTV
{'No internet service', 'Yes', 'No'}
StreamingMovies
{'No internet service', 'Yes', 'No'}
Contract
{'One year', 'Month-to-month', 'Two year'}
PaperlessBilling
{'Yes', 'No'}
PaymentMethod
{'Credit card (automatic)', 'Bank transfer (automatic)', 'Electronic check', 'Mailed check'}


In [36]:
# 1 if male, 0 otherwise
data['gender'] = (data['gender']=='Male')*1

In [79]:
# 1 if internet, 0 otherwise
data['Internet'] = (~(data['InternetService'] == 'No'))*1
# 1 if fiber, 0 if DSL
data['Fiber'] = (data['InternetService'] == 'Fiber optic')*1

# Now can drop 'InternetService' feature because same information
data.drop(columns='InternetService', inplace=True)

In [52]:
# 0 -> month to month
# 1 -> 1 year
# 2 -> 2 year
data['Contract'] = pd.factorize(data['Contract'])[0]

In [55]:
# Electronic check          -> 0
# Mailed check              -> 1
# Bank transfer (automatic) -> 2
# Credit card (automatic)   -> 3
data['PaymentMethod'] = pd.factorize(data['PaymentMethod'])[0]

In [81]:
# Deal with the rest of categorical variable
# Yes -> 1
# No  -> 0
for column in data.select_dtypes(include=np.object).columns:
    data[column] = (data[column]=='Yes')*1
    

In [83]:
data.dtypes

gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float32
MonthlyExpense      float64
Fiber                 int64
Internet              int64
dtype: object

## Feature selection

### Uivariate selection
Univariate linear regression tests.
Linear model for testing the individual effect of each of many regressors.
This is done in 2 steps:
    * The correlation between each regressor and the target is computed, that is, ((X[:, i] - mean(X[:, i])) * (y - mean_y)) / (std(X[:, i]) * std(y)).
    * It is converted to an F score then to a p-value.


In [166]:
from sklearn.feature_selection import f_regression

y = data['MonthlyExpense']  #independent variable
x = data.drop(columns=['MonthlyExpense'])    #feature

F_val, p_val = f_regression(x,y)

bestfeatures = pd.DataFrame(index=x.columns)
bestfeatures['F_value'] = F_val
bestfeatures['p_value'] = p_val

bestfeatures.sort_values('F_value',ascending=False)

### Feature importance

In [131]:
from sklearn.ensemble import RandomForestRegressor

x = data.drop(columns=['MonthlyExpense','MonthlyCharges']) 

model = RandomForestRegressor()
model.fit(x, y)

feat_importances = pd.DataFrame(model.feature_importances_, index=x.columns)


feat_importances.plot(kind='barh',legend=False,figsize=(12,8))

plt.show()

### Correlation matrix of features

In [150]:
import seaborn as sns

plt.figure(figsize=(20,20))
g=sns.heatmap(data.corr(),annot=True,cmap='BuPu')